In [2]:
import sys
from pathlib import Path
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib
import geopandas as gpd
# import osmnx as ox
import seaborn as sns
import gtfs_kit as gk
%load_ext autoreload
%autoreload 2
%matplotlib inline

# list gtfs paths
March_May_GTFS = '../../data/raw/PVTA-GTFS/gtfs-15-march-2021.zip'
June_August_GTFS = '../../data/raw/PVTA-GTFS/gtfs-24-june-2021.zip'
Jan_march_GTFS='../../data/raw/PVTA-GTFS/gtfs-26-jan-2021.zip'
Jan_2022_GTFS='../../data/raw/PVTA-GTFS/gtfs-19-jan-2022.zip'
dec_22_2021_GTFS='../../data/raw/PVTA-GTFS/gtfs-22-dec-2021.zip'
dec_14_2021_GTFS='../../data/raw/PVTA-GTFS/gtfs-14-dec-2021.zip'

--------
# Settings
## set which gtfs file is needed

In [3]:
x = input('Which period:\n'
          '1: March-May\n'
          '2: June-July\n'
          "3: Jan-March\n"
          "4: Jan-2022\n"
          "5: dec-22-2021\n"
          "6: dec-14-2021\n")
if x == '1':
    path = March_May_GTFS
elif x == '2':
    path = June_August_GTFS
elif x == '3':
    path = Jan_march_GTFS
elif x == '4':
    path = Jan_2022_GTFS
elif x == '5':
    path = dec_22_2021_GTFS
elif x == '6':
    path = dec_14_2021_GTFS
print(path)

../../data/raw/PVTA-GTFS/gtfs-26-jan-2021.zip


# Start program

In [4]:
# General information
# List feed
# gk.list_feed(path)
feed = gk.read_feed(path, dist_units='km')
display('Chosen file discription',feed.describe())
# Validate
display('Chosen file validation',feed.validate())

# Append shape_dist_traveled column to stop times
# display(feed.stop_times.head().T.T)
#
# feed = feed.append_dist_to_stop_times()
# feed.stop_times.head().T.T

'Chosen file discription'

,indicator,value
0,agencies,[PVTA]
1,timezone,America/New_York
2,start_date,20210122
3,end_date,20210320
4,num_routes,36
5,num_trips,5422
6,num_stops,1703
7,num_shapes,354
8,sample_date,20210128
9,num_routes_active_on_sample_date,33


'Chosen file validation'

,type,message,table,rows
0,warning,Feed expired,calendar/calendar_dates,[]


*************************
# Build a route timetable and save it to be used in script 1

# Use parquet to save and load these


In [5]:
# Choose study dates
# week = feed.get_first_week()
dates = feed.get_dates() # get a list of dates for which this GTFS is valid
# dates

## For single route
# route_id = feed.routes['route_id'].iat[0] # only one route can be displayed
# route_id
# feed.build_route_timetable(route_id, dates)#.T # reformat date to date_time. Then you have time, date, and trip and route ID in the same table

## For all routes
# Iterate over all routes and append these together

route_timetable = pd.DataFrame()

for route_id in feed.routes['route_id'].unique():
    route_timetable_ = feed.build_route_timetable(route_id, dates)
    route_timetable = route_timetable.append(route_timetable_)

route_timetable = route_timetable.drop(columns = ['shape_dist_traveled','drop_off_type','pickup_type','stop_sequence','direction_id','block_id','shape_id','service_id'])

route_timetable['date'] = route_timetable['date'].astype('int64')
route_timetable['date'] = pd.to_datetime(route_timetable['date'],format = '%Y%m%d')



if path == March_May_GTFS:
    route_timetable.to_csv(r"../../data/raw/PVTA-GTFS/route_timetable_March_May.csv",index=False)
if path == June_August_GTFS:
    route_timetable.to_csv(r"../../data/raw/PVTA-GTFS/route_timetable_June_August.csv",index=False)

if path == Jan_march_GTFS:
    route_timetable.to_csv(r"../../data/raw/PVTA-GTFS/route_timetable_Jan_March.csv",index=False)

if path == Jan_2022_GTFS:
    route_timetable.to_csv(r"../../data/raw/PVTA-GTFS/route_timetable_Jan_2022.csv",index=False)
if path == dec_22_2021_GTFS:
    route_timetable.to_csv(r"../../data/raw/PVTA-GTFS/route_timetable_dec_22_2021.csv",index=False)

if path == dec_14_2021_GTFS:
    route_timetable.to_csv(r"../../data/raw/PVTA-GTFS/route_timetable_dec_14_2021.csv",index=False)

# route_timetable = pd.read_csv(r"../../data/raw/PVTA-GTFS/route_timetable_dec_2021.csv",low_memory=False)
route_timetable # All routes from the whole gtfs duration

C:\Users\mazee\AppData\Local\Temp\ipykernel_20148\2731941876.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  route_timetable = route_timetable.append(route_timetable_)
C:\Users\mazee\AppData\Local\Temp\ipykernel_20148\2731941876.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  route_timetable = route_timetable.append(route_timetable_)
C:\Users\mazee\AppData\Local\Temp\ipykernel_20148\2731941876.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  route_timetable = route_timetable.append(route_timetable_)
C:\Users\mazee\AppData\Local\Temp\ipykernel_20148\2731941876.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  route_timet

,route_id,trip_id,trip_headsign,arrival_time,departure_time,stop_id,date
117983,30,3411651-UM2021-UMTS-Weekday-40,Old Belchertown Road,07:15:00,07:15:00,34,2021-01-22
118963,30,3411702-UM2021-UMTS-Weekday-40,North Amherst,07:15:00,07:15:00,148,2021-01-22
117984,30,3411651-UM2021-UMTS-Weekday-40,Old Belchertown Road,07:15:41,07:15:41,35,2021-01-22
118964,30,3411702-UM2021-UMTS-Weekday-40,North Amherst,07:15:32,07:15:32,8002,2021-01-22
117985,30,3411651-UM2021-UMTS-Weekday-40,Old Belchertown Road,07:16:38,07:16:38,38,2021-01-22
...,...,...,...,...,...,...,...
78807,X92,3424756-S2012-AMD-Saturday-50,Memorial Industrial Park via Roosevelt,17:44:24,17:44:24,774,2021-03-20
78808,X92,3424756-S2012-AMD-Saturday-50,Memorial Industrial Park via Roosevelt,17:44:56,17:44:56,778,2021-03-20
78809,X92,3424756-S2012-AMD-Saturday-50,Memorial Industrial Park via Roosevelt,17:45:32,17:45:32,795,2021-03-20
78810,X92,3424756-S2012-AMD-Saturday-50,Memorial Industrial Park via Roosevelt,17:45:57,17:45:57,803,2021-03-20


## combine timetables (goal is to get all of them in one file)

In [6]:
route_timetable_1 = pd.read_csv(r"../../data/raw/PVTA-GTFS/route_timetable_dec_2021.csv",low_memory=False)
route_timetable_2 = pd.read_csv(r"../../data/raw/PVTA-GTFS/route_timetable_Jan_2022.csv",low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/raw/PVTA-GTFS/route_timetable_Jan_2022.csv'

In [ ]:
route_timetable_1.head()

In [ ]:
route_timetable_2.head()

In [6]:
if path == Jan_march_GTFS:
    route_timetable.to_csv(r"../../data/raw/PVTA-GTFS/route_timetable_Jan_March.csv",index=False)

----------------------------------
# Build a stop timetable (not important)

In [ ]:
stop_id = feed.stops['stop_id'].iat[0]
# stop_id
feed.build_stop_timetable(stop_id, dates)

In [ ]:
# Iterate over all stops and append these together

stop_timetable = pd.DataFrame()

for stop_id in feed.stops['stop_id'].unique():
    stop_timetable_ = feed.build_stop_timetable(stop_id, dates)
    stop_timetable = stop_timetable.append(stop_timetable_)

stop_timetable # all routes

In [ ]:
stop_timetable.drop(columns = ['shape_dist_traveled','drop_off_type','pickup_type','stop_sequence','direction_id','block_id','shape_id','service_id'])
stop_timetable

***
***
# Non essential

In [ ]:
# Compute feed time series
trip_stats = feed.compute_trip_stats()
trip_stats.head().T
fts = feed.compute_feed_time_series(trip_stats, dates, freq='6H')
fts

In [ ]:
gk.downsample(fts, freq='12H')

In [ ]:
# Compute feed stats for first week

feed_stats = feed.compute_feed_stats(trip_stats, week)
feed_stats

In [ ]:
# Compute route time series

rts = feed.compute_route_time_series(trip_stats, dates[4:11], freq='4H')
# rts

In [ ]:
rts_ = rts['num_trip_ends']
# rts_ = rts_.melt(30, var_name='cols', value_name='vals')
rts_ = rts_.stack()
rts_ = rts_.reset_index()#.set_index('datetime')
rts_

In [ ]:
plt.figure(figsize = (60,15))

sns.lineplot(y = 0, x = 'datetime',data = rts_, hue = 'route_id')


In [ ]:
# Slice time series

# inds = ['service_distance', 'service_duration', 'service_speed']
# rids = ['110-423', '110N-423']

# rts.loc[:, (inds, rids)]

In [ ]:
# Slice again by cross-section

# rts.xs(rids[0], axis="columns", level=1)

In [ ]:
# Compute trip locations for every hour

rng = pd.date_range('1/1/2000', periods=24, freq='H')
times = [t.strftime('%H:%M:%S') for t in rng]
loc = feed.locate_trips(dates[0], times)
loc.head()





In [ ]:
# Compute screen line counts

# path = DATA_DIR/'cairns_screen_lines.geojson'
# lines = gp.read_file(path)

# display(lines)

# feed.compute_screen_line_counts(lines, dates)

In [ ]:
# Map routes

rids = feed.routes.route_id#.loc[2:4]


feed.map_routes(rids, include_stops=False)

***
# Map stops

In [ ]:
stp = feed.stops.stop_id

feed.STOP_STYLE = {'color': '#fc8d62', 'fill': 'false', 'fillOpacity': 0.7, 'radius': 1, 'weight': 0.5}

feed.map_stops(stp)

In [ ]:
d = {'StopID': [1,6,5,0,1,1],"col2":list("cabass")}
df = pd.DataFrame(data=d)
df

In [ ]:
df = df.loc[df.StopID.shift(-1) != df.StopID]

df

In [ ]:

df["NextBoardingStop"]= df["StopID"].shift(-1)
df['NextBoardingStop'].iloc[-1] = np.array(df.StopID.head(1))[0]
df

In [ ]:
if df.NextBoardingStop.iloc[-1] == df.StopID.iloc[-1]:
        df.drop(df.tail(1).index,inplace=True) # drop last row in case last activation is the same as the first (assumption: the last destination is actually the first one. But accidental activation has been done)

In [ ]:
df